In [0]:
import torch

In [0]:
from torchsummary import summary
import pandas as pd
import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage

In [0]:
encoder = {
    'happy': 0,
    'surprised' : 1,
    'angry': 2,
    'sad': 3,
    'Unknown': 4

}
decoder = {
    0 : 'happy',
    1 : 'surprised' ,
    2 : 'angry',
    3 : 'sad',
    4 : 'Unknown'

}

In [0]:
import pandas as pd
file_path_test = "/content/Train.csv"
root_dir_test = "/content"
csv_file = pd.read_csv(file_path_test)
csv_file.head()

,Frame_ID,Emotion
0,frame0.jpg,happy
1,frame1.jpg,happy
2,frame2.jpg,surprised
3,frame3.jpg,surprised
4,frame4.jpg,angry


In [0]:
print(len(csv_file))

298


In [0]:
import pandas as pd
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
train_img = []
train_aug = []

transformation = transforms.Compose([
                                      transforms.ToPILImage(),
                                      transforms.Resize((224,224), interpolation=2),
                                      transforms.ToTensor()
                                    ])
for i in range(0 , len(csv_file)):
  frame = csv_file.iloc[i,0]
  label = csv_file.iloc[i,1]
  image = root_dir_test  + '/' + frame
  img =  cv2.imread(image)
  train_aug.append(img)
  imagetensor = transformation(img)
  imagenumpy = imagetensor.numpy()
  imagenew = imagenumpy/255
  train_img.append(imagenew)
 
  

train_x = np.array(train_img)

train_y = csv_file['Emotion'].values
train_x.shape, train_y.shape
  

((298, 3, 224, 224), (298,))

In [0]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)
def augment_flips_color(p=.5):
    return Compose([
        CLAHE(),
        RandomRotate90(),
        Transpose(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
        Blur(blur_limit=3),
        OpticalDistortion(),
        GridDistortion(),
        HueSaturationValue()
    ], p=p)

    
def augment_and_show(aug, image):
    image = aug(image=image)['image']
    return image

In [0]:
train_aug_new = []
aug_final = []
for i in range( 0,  298):
  image = train_aug[i]
  aug = augment_flips_color(p=1)
  x = augment_and_show(aug, image)
  imagetensor = transformation(x)
  imagenumpy = imagetensor.numpy()
  imagenew = imagenumpy/255
  train_aug_new.append(imagenew)

aug_final = np.array(train_aug_new)



In [0]:
final_train_data = []
final_target_train = []

for i in tqdm(range(train_x.shape[0])):
    final_train_data.append(train_x[i])
    final_train_data.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_train_data.append(np.fliplr(train_x[i]))
    final_train_data.append(np.flipud(train_x[i]))
    final_train_data.append(random_noise(train_x[i],var=0.2**2))
    final_train_data.append(aug_final[i])
     
    for j in range(6):
        final_target_train.append(encoder[train_y[i]])

100%|██████████| 298/298 [00:07<00:00, 41.99it/s]


In [0]:
len(final_target_train), len(final_train_data)
final_train = np.array(final_train_data)
final_target_train = np.array(final_target_train)
print(len(final_train))
print(len(final_target_train))

1788
1788


In [0]:

import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [0]:
final_train = final_train.reshape(1788, 3, 224, 224)
final_train  = torch.from_numpy(final_train)
final_train = final_train.float()

# converting the target into torch format
final_target_train = final_target_train.astype(int)
final_target_train = torch.from_numpy(final_target_train)

In [0]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
        )

        self.linear_layers = Sequential(
            Linear(128 * 14 * 14, 512),
            ReLU(inplace=True),
            Dropout(),
            Linear(512, 256),
            ReLU(inplace=True),
            Dropout(),
            Linear(256,10),
            ReLU(inplace=True),
            Dropout(),
            Linear(10,5)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [0]:
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.000075)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

#print(model)

In [0]:
torch.manual_seed(0)

# batch size of the model
batch_size = 128

# number of epochs to train the model
n_epochs = 100

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
        
    permutation = torch.randperm(final_train.size()[0])

    training_loss = []
    for i in tqdm(range(0,final_train.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = final_train[indices], final_target_train[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

  7%|▋         | 1/14 [00:00<00:02,  5.17it/s]

epoch: 	 1 	 training loss: 	 0.29613408020564486


  7%|▋         | 1/14 [00:00<00:02,  5.19it/s]

epoch: 	 2 	 training loss: 	 0.29228564351797104


  7%|▋         | 1/14 [00:00<00:02,  5.21it/s]

...

epoch: 	 98 	 training loss: 	 0.25023029425314497


  7%|▋         | 1/14 [00:00<00:02,  5.24it/s]

epoch: 	 99 	 training loss: 	 0.24025798908301763


100%|██████████| 14/14 [00:02<00:00,  4.96it/s]

epoch: 	 100 	 training loss: 	 0.25096918642520905


In [0]:
PATH = '/content/model.pth'
torch.save(model , PATH)
the_model= torch.load(PATH)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
torch.manual_seed(0)
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(final_train.size()[0])
for i in tqdm(range(0,final_train.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = final_train[indices], final_target_train[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

100%|██████████| 14/14 [00:01<00:00, 11.71it/s]

training accuracy: 	 0.8416798675115207


In [0]:
#Testing File
from PIL import Image
import matplotlib.pyplot as plt
file_path_test = "/Test.csv"
root_dir_test = "/"
i = 0
result = []
image_file = []
csv_file_test = pd.read_csv(file_path_test)
for i in range(0,len(csv_file_test)):
  
  frame = csv_file_test.iloc[i,0]
  image = root_dir_test  + '/' + frame
  imagetensor = cv2.imread(image)
  #img = Image.open(image)
  #plt.imshow(img)
  to_pil = transforms.Compose([transforms.ToPILImage(),transforms.CenterCrop(700)])
  images = imagetensor
  fig=plt.figure(figsize=(10,10))
  image = to_pil(images)
  #plt.imshow(image)
  index = predict_image(image)
  image_file.append(frame)
  result.append(decoder[index])

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = image_tensor
    input = input.to('cuda')
    output = the_model(input)
    index = output.data.cpu().numpy().argmax()
    return index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [0]:
dict = {'Frame_ID': image_file, 'Emotion': result}
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv('file27.csv') 
df

,Frame_ID,Emotion
0,test0.jpg,Unknown
1,test1.jpg,surprised
2,test2.jpg,happy
3,test3.jpg,Unknown
4,test4.jpg,surprised
...,...,...
181,test181.jpg,angry
182,test182.jpg,angry
183,test183.jpg,surprised
184,test184.jpg,angry
